# 0. Import/Classes/

In [177]:
from bs4 import BeautifulSoup as BS
import undetected_chromedriver as uc
import json
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException, ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException, ElementNotInteractableException
import re
from time import sleep as time_sleep, time
from queue import Queue
import math
from typing import NamedTuple
from datetime import datetime
from zipfile import ZipFile, ZIP_DEFLATED
from selenium import webdriver
import requests
from webdriver_manager.chrome import ChromeDriverManager
import lxml
from itertools import chain
from typing import NamedTuple
import pickle
from collections import deque

In [178]:
class market_link(NamedTuple):
    sity: str
    title: str
    discounts: int
    alcohol: bool
    href: str
    page_href: str

class MyTimeoutErrore(Exception):
    def __init__(self, text):
        self.txt = text

In [179]:
driver = webdriver.Chrome()

# 1. Получение адресов городов

# 2. Получение ссылок на Магазины

## 2.1. Получение ссылки на старицы каталогов с магазинами

### 2.1.1. Функции

In [140]:
def _get_page_count(drive: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        # wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            print(ex)
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            print(ex)
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise Exception("timeout")
    
    return out_list

### 2.1.2. Выполнение

In [5]:
# driver.get('https://edadeal.ru/lipeck/retailers')
url = 'https://edadeal.ru/lipeck/retailers'

In [98]:
temp_url_list = _get_page_count(driver, url, '?page=')
temp_url_list

['https://edadeal.ru/lipeck/retailers?page=1',
 'https://edadeal.ru/lipeck/retailers?page=2',
 'https://edadeal.ru/lipeck/retailers?page=3',
 'https://edadeal.ru/lipeck/retailers?page=4',
 'https://edadeal.ru/lipeck/retailers?page=5']

## 2.2. Получение непосредственно все ссылки на Магазины

### 2.2.1. Функции

In [141]:
def _get_retailers_links(drive: webdriver.Chrome, url: str, sity: str, timeout: int=30) -> list[market_link]:
    """
        Возвращает лист с магазинами и ссылками.
    """
    
    host = '/'.join(url.split('/')[:-1])
    out_list = list()
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout + 10,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )

    
    # Вызов страницы
    driver.get(url)
    time_start = int(time())
    
    while (int(time()) - time_start) < timeout:
        # Задержка действий до загрузки отслеживаемых елементов.
        wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@aria-label='Постраничная навигация']"))
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('div', class_='p-dsk-srch-retailers__content').find('div', class_='b-dsk-grid__container').find_all('article')
        except Exception as ex:
            print(ex)
            continue
        
        try:    
            for item in f1:
                href = host + item.find('a').get('href')
                title = item.find('a').find('h4').get('title')
                discounts = int((re.findall(r'(\d+)', item.find('a').find('p').text)[0]))
                out_list.append(market_link(sity, title, discounts, False, href, None))
                
        except Exception as ex:
            print(ex)
            continue
        
        break
    
    else:
        raise MyTimeoutErrore('timeout')
    
    return out_list

### 2.2.2. Выполнение

In [100]:
temp_retailer_links = list()
for link in temp_url_list:
    temp_retailer_links.append(_get_retailers_links(driver, link, 'lipetsk'))

retailer_links = list(chain.from_iterable(temp_retailer_links))
print(len(retailer_links))
print(retailer_links)

55
[market_link(sity='lipetsk', title='Магнит', discounts=10873, alcohol=False, href='https://edadeal.ru/lipeck/retailers/magnit-univer', page_href=None), market_link(sity='lipetsk', title='Пятёрочка', discounts=30, alcohol=False, href='https://edadeal.ru/lipeck/retailers/5ka', page_href=None), market_link(sity='lipetsk', title='Перекрёсток', discounts=1877, alcohol=False, href='https://edadeal.ru/lipeck/retailers/perekrestok', page_href=None), market_link(sity='lipetsk', title='Магнит Косметик', discounts=10498, alcohol=False, href='https://edadeal.ru/lipeck/retailers/magnit-cosmo', page_href=None), market_link(sity='lipetsk', title="О'кей гипермаркет", discounts=851, alcohol=False, href='https://edadeal.ru/lipeck/retailers/okmarket-giper', page_href=None), market_link(sity='lipetsk', title='Metro', discounts=6079, alcohol=False, href='https://edadeal.ru/lipeck/retailers/metro-cc', page_href=None), market_link(sity='lipetsk', title='Лента Гипермаркет', discounts=5796, alcohol=False, h

### 2.2.3. Save/Load

In [101]:
with open('data/retailer_links.pickle', 'wb') as file:
    pickle.dump(retailer_links, file)

In [156]:
with open('data/retailer_links.pickle', 'rb') as file:
    retailer_links = pickle.load(file)

# 3. Получение адресов Товаров

## 3.1. Получение ссылок на каталоги с товарами

### 3.1.1. Функции

In [168]:
def _get_page_count2(drive: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> tuple[list[str],bool]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    try:
        driver.get(url)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    alcohol = True
    # Проверка появления "Да, мне есть 18"
    try:
        button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
        button.click()
    except TimeoutException:
        alcohol = False
    except ElementNotInteractableException:
        alcohol = False
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol)

def _get_all_links_to_market(drive: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение адресов на все страницы с каталогами товаров
        Что бы после можно было использовать много поточность.
    """
    loger_fault = list()
    # Очередь
    q = deque(list_url)
    # Нужно вывести дейдлайн, который позволит вывести обработаные данные, и не обработанные. ???????????
    out_list = list()
    while len(q) > 0:
        item = q.popleft()
        
        if item.discounts > 24:
            try:
                temp_url_list, alcohol = _get_page_count2(driver, item.href, '?page=', timeout)
            except MyTimeoutErrore as mte:
                q.append(item)
                loger_fault.append(mte)
                continue
            
            for href in temp_url_list:
                out_list.append(market_link(item.sity,
                                            item.title,
                                            item.discounts,
                                            alcohol,
                                            item.href,
                                            href))
                
        elif item.discounts:
            out_list.append(market_link(item.sity,
                                        item.title,
                                        item.discounts,
                                        True,
                                        item.href,
                                        item.href))
        else:
            out_list.append(market_link(*item[:-1], None))

    return (out_list, loger_fault)

### 3.1.2. Выполнение

In [159]:
# Получение всех адресов на страницы с товарами.
retailer_links_pages, loger_fault = _get_all_links_to_market(driver, retailer_links, timeout=10)
len(retailer_links_pages)

931

In [162]:
loger_fault

[]

In [163]:
retailer_links_pages

[market_link(sity='lipetsk', title='Магнит', discounts=10873, alcohol=True, href='https://edadeal.ru/lipeck/retailers/magnit-univer', page_href='https://edadeal.ru/lipeck/retailers/magnit-univer?page=1'),
 market_link(sity='lipetsk', title='Магнит', discounts=10873, alcohol=True, href='https://edadeal.ru/lipeck/retailers/magnit-univer', page_href='https://edadeal.ru/lipeck/retailers/magnit-univer?page=2'),
 market_link(sity='lipetsk', title='Магнит', discounts=10873, alcohol=True, href='https://edadeal.ru/lipeck/retailers/magnit-univer', page_href='https://edadeal.ru/lipeck/retailers/magnit-univer?page=3'),
 market_link(sity='lipetsk', title='Магнит', discounts=10873, alcohol=True, href='https://edadeal.ru/lipeck/retailers/magnit-univer', page_href='https://edadeal.ru/lipeck/retailers/magnit-univer?page=4'),
 market_link(sity='lipetsk', title='Магнит', discounts=10873, alcohol=True, href='https://edadeal.ru/lipeck/retailers/magnit-univer', page_href='https://edadeal.ru/lipeck/retailers

### 3.1.3. Save/Load

In [160]:
with open('data/retailer_links_pages.pickle', 'wb') as file:
    pickle.dump(retailer_links_pages, file)

In [180]:
with open('data/retailer_links_pages.pickle', 'rb') as file:
    retailer_links_pages = pickle.load(file)

## 3.2. Получение информации о товарах и ссылки на них

### 3.2.1 Функции

In [167]:
def _get_page_count2(drive: webdriver.Chrome, url: str, pagin_sufix: str, timeout: int=30) -> list[str]:
    """
        Возвращает лист с доступными ссылками
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    # Вызов страницы
    try:
        driver.get(url)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    alcohol = True
    # Проверка появления "Да, мне есть 18"
    try:
        button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
        button.click()
    except TimeoutException:
        alcohol = False
    except ElementNotInteractableException:
        alcohol = False
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol)

def _get_all_links_to_market(drive: webdriver.Chrome, list_url: list[market_link], timeout: int=30) -> list[market_link]:
    """
        Получение адресов на все страницы с каталогами товаров
        Что бы после можно было использовать много поточность.
    """
    loger_fault = list()
    # Очередь
    q = deque(list_url)
    # Нужно вывести дейдлайн, который позволит вывести обработаные данные, и не обработанные. ???????????
    out_list = list()
    while len(q) > 0:
        item = q.popleft()
        
        if item.discounts > 24:
            try:
                temp_url_list, alcohol = _get_page_count2(driver, item.href, '?page=', timeout)
            except MyTimeoutErrore as mte:
                q.append(item)
                loger_fault.append(mte)
                continue
            
            for href in temp_url_list:
                out_list.append(market_link(item.sity,
                                            item.title,
                                            item.discounts,
                                            alcohol,
                                            item.href,
                                            href))
                
        elif item.discounts:
            out_list.append(market_link(item.sity,
                                        item.title,
                                        item.discounts,
                                        True,
                                        item.href,
                                        item.href))
        else:
            out_list.append(market_link(*item[:-1], None))

    return (out_list, loger_fault)



def _get_sku_info(drive: webdriver.Chrome, item: market_link, timeout: int=30):
    """
        Возвращает что-то об продукте.
    """
    # Класс задержки, который будет ожидать когда догрузится элемент
    wait = WebDriverWait(
        driver,
        timeout=timeout,
        poll_frequency=1,
        # ignored_exceptions=[ElementNotVisibleException, ElementNotSelectableException, NoSuchElementException]
        )
    
    # Вызов страницы
    try:
        driver.get(item.page_href)
    except TimeoutException as ex:
        raise MyTimeoutErrore(f'timeout bad DRIVER get url: {url}')
    
    # Клик по кнопке "Есть 18 лет"
    if item.alcohol:
        # Проверка появления "Да, мне есть 18"
        try:
            button = wait.until(lambda d: d.find_element(by=By.XPATH, value="//div[@class='b-dsk-adult-disclaimer__wrapper']//button[@type='button']"))
            button.click()
        except TimeoutException:
            pass
        except ElementNotInteractableException:
            pass                                                                                # Нужно добавить логирование
    
    time_start = int(time())
    while (int(time()) - time_start) < timeout*2:
        # Задержка действий до загрузки отслеживаемых елементов.
        try:
            wait.until(lambda d: d.find_element(by=By.XPATH, value="//nav[@data-test-ref='paginator']"))
        except TimeoutException:
            raise MyTimeoutErrore(f'timeout not TABLE in url: {url}')
                
        # Парсинг
        page = BS(driver.page_source, 'lxml')
        
        try:
            f1 = page.find('nav', {'data-test-ref':"paginator"})
        except Exception as ex:
            continue
            
        try:    
            pagination = int(f1.find_all('a')[-2].text)
        except Exception as ex:
            continue
        
        out_list = [f'{url}{pagin_sufix}{x}' for x in range(1, pagination+1)]
        
        break
    else:
        raise MyTimeoutErrore(f'timeout not PAGINATOR in url: {url}')
    
    return (out_list, alcohol)
    

### 3.2.2. Выполнение

In [183]:
# retailer_links_pages
url='https://edadeal.ru/lipeck/retailers/sozvezdie-krasoty'
driver.get(url)
while True:
    try:
        driver.find_element(by=By.XPATH, value="//div[@data-test-ref='grid']")
    except Exception as ex:
        print('no')
        time_sleep(0.5)
    else:
        break
print('yes')

page = BS(driver.page_source, 'lxml')
f1 = page.find('div', {'data-test-ref':'grid'})

yes


In [189]:
f2 = f1.find('div').find_all('div', {'data-test-ref':'b-srch-card'})
len(f2)

21